# 평가
1. 내부규정 평가
    - 청크보완 방식 : 슬라이딩 윈도우 방식
    - 임베딩 모델 : KURE-V1
    - 리랭크 모델 : BGE-RERANKER-KO
    - 질문지 : GPT-4O 생성 132개질문지
    - LLM : GPT-4O-mini
    - 프롬프트 5종에 따른 점수평가
2. 평가 결과
    - 청크보완임베딩 + gpt - f1/em  : 40.4433/0.000040
    - 청크보완임베딩 + gpt + 리랭커 + 금지형 프롬프트 - f1/em : 55.04/10.61
    - 청크보완임베딩 + gpt + 리랭커 + 4가지 프롬프트 - f1/em 
      : 43.17/0.76 47.58/6.82 46.91/0.00 14.6/00.00


In [1]:
# 라이브러리 및 환경설정

from openai import OpenAI
import json
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import evaluate

client = OpenAI(api_key="Key")  

# 임베딩모델 로드 ( 허깅페이스 )
embedding = HuggingFaceEmbeddings(
    model_name="work1/models/kure_v1",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)


c:\Users\kdy\anaconda3\envs\llm3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\kdy\AppData\Local\Temp\ipykernel_23076\794522863.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(


In [8]:
# 1. 청크보완임베딩 + gpt - f1/em  : 40.4433/0.000040

import json
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import evaluate

# 1. 임베딩 모델 로드
embedding = HuggingFaceEmbeddings(
    model_name="work1/models/kure_v1",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# 2. FAISS 벡터 DB 로드
vectorstore = FAISS.load_local(
    folder_path="faiss_win",
    embeddings=embedding,
    index_name="index",
    allow_dangerous_deserialization=True
)

# 3. 평가 질문지 로드
with open("eval_questions_window.jsonl", "r", encoding="utf-8") as f:
    qa_pairs = [json.loads(line) for line in f]

# 4. GPT-4o 호출 함수 (순차 처리, Top-k=1)
import asyncio
import nest_asyncio
nest_asyncio.apply()

# GPT-4o 호출 함수 (동기 버전)
def ask(question, context):
    messages = [
        {"role": "system", "content": "당신은 회사 규정에 대해 정확하게 답변하는 도우미입니다."},
        {"role": "user", "content": f"다음은 관련 문서입니다:\n{context}"},
        {"role": "user", "content": f"질문: {question}"}
    ]
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0,
        max_tokens=512
    )
    return response.choices[0].message.content.strip()

# 전체 실행 루프 (동기 처리)
results = []
for qa in qa_pairs:
    docs = vectorstore.similarity_search(qa["question"], k=1)
    context = docs[0].page_content if docs else ""
    gen = ask(qa["question"], context)
    results.append(gen)


# 5. SQuAD 평가
squad = evaluate.load("squad")
predictions = [{"id": str(i), "prediction_text": gen} for i, gen in enumerate(results)]
references = [{"id": str(i), "answers": {"text": [qa["answer"]], "answer_start": [0]}} for i, qa in enumerate(qa_pairs)]

score = squad.compute(predictions=predictions, references=references)
print(f"📊 F1 평균: {score['f1']:.4f}")
print(f"📊 EM 평균: {score['exact_match']:.4f}")


📊 F1 평균: 40.4433
📊 EM 평균: 0.0000


In [ ]:
# 청크보완임베딩 + gpt + 리랭커 + 금지형 프롬프트 - f1/em : 55.04/10.61

import json
from tqdm import tqdm
from sentence_transformers import CrossEncoder
from langchain_community.vectorstores import FAISS
from evaluate import load
from openai import OpenAI

# 🔹 이미 선언된 객체들 (필요시 수정)
embedding = HuggingFaceEmbeddings(
    model_name="work1/models/kure_v1",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# 1. 리랭커 로드 (bge-reranker-v2-m3-ko)
reranker = CrossEncoder("work1/models/bge-reranker-v2-m3-ko", device="cuda")

# 2. FAISS 벡터DB 로드
vectorstore = FAISS.load_local(
    folder_path="faiss_win",
    embeddings=embedding,
    index_name="index",
    allow_dangerous_deserialization=True
)

# 3. 평가 질문 로드
with open("eval_questions_window.jsonl", "r", encoding="utf-8") as f:
    eval_questions = [json.loads(line) for line in f]

# 4. 평가 지표 준비
f1 = load("evaluate-metric/squad", "f1")
em = load("evaluate-metric/squad", "exact_match")

f1_scores, em_scores = [], []

# 5. 평가 루프
for idx, qa in tqdm(enumerate(eval_questions), total=len(eval_questions)):
    query = qa["question"]
    answer = qa["answer"]
    qid = qa.get("id", str(idx))  # ID 없으면 인덱스 사용

    # Step 1: Top-5 문서 검색
    docs = vectorstore.similarity_search(query, k=5)

    # Step 2: reranker로 점수 계산
    reranker_inputs = [[query, doc.page_content] for doc in docs]
    scores = reranker.predict(reranker_inputs)

    # Step 3: 점수 기반 정렬
    reranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
    top_doc = reranked[0][0].page_content  # 가장 높은 점수 1개 사용

    # Step 4: GPT-4o 호출 (프롬프트 개선 포함)
    system_prompt = (
        "너는 회사의 사내 규정을 정확히 안내하는 QA 비서야. "
        "다음 문서의 내용에 기반해서만 답변해. "
        "문서에 없는 내용은 추론하지 말고 '문서에 없습니다'라고 답변해."
    )
    user_prompt = f"""문서:
{top_doc}

질문: {query}"""

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    prediction = completion.choices[0].message.content.strip()

    # Step 5: 평가 포맷 변환 및 F1/EM 계산
    prediction_dict = {"id": qid, "prediction_text": prediction}
    reference_dict = {
        "id": qid,
        "answers": [{"text": answer, "answer_start": 0}]
    }

    f1_score = f1.compute(predictions=[prediction_dict], references=[reference_dict])["f1"]
    em_score = em.compute(predictions=[prediction_dict], references=[reference_dict])["exact_match"]

    f1_scores.append(f1_score)
    em_scores.append(em_score)

# 6. 결과 출력
print(f"📊 평균 F1: {sum(f1_scores)/len(f1_scores):.2f}")
print(f"📊 평균 EM: {sum(em_scores)/len(em_scores):.2f}")


100%|██████████| 132/132 [08:21<00:00,  3.80s/it]

📊 평균 F1: 55.04
📊 평균 EM: 10.61


In [15]:
# 청크보완임베딩 + gpt + 리랭커 + 4가지 프롬프트 - f1/em 

from rag_eval import run_prompt_ab_test

run_prompt_ab_test(
    client=client,
    embedding=embedding,
    prompt_styles=["basic", "strict", "cot", "step"],  # 실험할 프롬프트 종류
    top_k=1
)



🔍 실험 프롬프트: basic
📊 평균 F1: 43.17
📊 평균 EM: 0.76

🔍 실험 프롬프트: strict
📊 평균 F1: 47.58
📊 평균 EM: 6.82

🔍 실험 프롬프트: cot
📊 평균 F1: 46.91
📊 평균 EM: 0.00

🔍 실험 프롬프트: step
📊 평균 F1: 14.60
📊 평균 EM: 0.00
